In [1]:
from astropy.io import fits
from astropy.table import Table
import pandas as pd
import nested_pandas as npd
import pyarrow as pa
import numpy as np

import tarfile
import glob

In [21]:
# untarring
filepath = "/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/"

ss_tarfiles = glob.glob(filepath+"ss_tar_26/*.tar.gz")


for ss_tarfile in ss_tarfiles[0:5]:
    file = tarfile.open(ss_tarfile) 
  
    # extracting file
    for member in file.getmembers():
        if "spPlate" in member.name:
            file.extract(member, filepath+"ss_26/")
#file.extractall(filepath+"ss_26/") 
  
    file.close()


In [3]:
# parquet-ifying
filepath = "/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26/"

ss_fitsfiles = glob.glob(filepath+"/*/*.fits")

for ss_fitsfile in ss_fitsfiles:
    obj_dat = Table.read(ss_fitsfile, hdu=5, format='fits')
    obj_dat.write('h5_pq/plate.parquet', format='parquet', overwrite=True)
    #obj_dat.to_pandas().to_parquet('h5_pq/plate.parquet')

    # Read in spFrame HDU data
    #with pd.read_parquet('h5_pq/plate.parquet', dtype_backend='pyarrow') as ndf:
    ndf = pd.read_parquet('h5_pq/plate.parquet', dtype_backend='pyarrow')
    #ndf = ndf.assign(OBJID_joined = lambda x: "".join(val for val in x["OBJID"]))
    #joined_objid = ndf.apply(lambda x: "".join([str(val) for val in x.OBJID.tolist()]), axis=1).astype("int")
    #ndf["joined_OBJID"] = joined_objid

    ndf = ndf.astype({"OBJID": pd.ArrowDtype(pa.list_(pa.int32())),
                      "MAG": pd.ArrowDtype(pa.list_(pa.float32())),
                      "CALIBFLUX": pd.ArrowDtype(pa.list_(pa.float32())),
                      "CALIBFLUX_IVAR": pd.ArrowDtype(pa.list_(pa.float32()))})
    with fits.open(ss_fitsfile) as spec:
        ndf["flux"] = pd.Series(spec[0].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["ivar"] = pd.Series(spec[1].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["andmask"] = pd.Series(spec[2].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
        ndf["ormask"] = pd.Series(spec[3].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
        ndf["wavedisp"] = pd.Series(spec[4].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
        ndf["skyflux"] = pd.Series(spec[6].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))

    ndf.to_parquet(f"/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/{ss_fitsfile.split('/')[-1].split('.')[0]}.parquet", 
    engine="pyarrow", compression="snappy", store_schema=False)






        Use textwrap.indent() instead. [astropy.io.fits.hdu.hdulist]
    Header size is not multiple of 2880: 1
There may be extra bytes after the last HDU or the file is corrupted. [astropy.io.fits.hdu.hdulist]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]
MC1TEMDN=-0.00000000000000E+00 / sp1 mech median temp                            [astropy.io.fits.card]
MC1TBCB =-0.00000000000000E+00 / sp1 mech Temp_Blue_Cam_Bot                      [astropy.io.fits.card]


In [29]:
#npd.read_parquet("/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/spPlate-0266-51602.parquet", dtype_backend="pyarrow")
#pd.read_parquet("h5_pq/plate.parquet", dtype_backend="pyarrow").dtypes

In [28]:
#hipscat importing
from hipscat_import.catalog.arguments import ImportArguments
import hipscat_import.pipeline as runner

args = ImportArguments(
    output_artifact_name="sdss_dr7_spectra",
    input_path="/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_pq/",
    file_reader="parquet",
    ra_column="RA",
    dec_column="DEC",
    sort_columns="joined_OBJID",
    pixel_threshold=1_000_000,
    highest_healpix_order=7,
    output_path="/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_catalog",
)
runner.pipeline(args)

/astro/users/brantd/.conda/envs/expr_310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Planning  : 100%|██████████| 5/5 [00:00<00:00,  7.09it/s]
/astro/users/brantd/.conda/envs/expr_310/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35225 instead
  warnings.warn(
Finishing : 100%|██████████| 5/5 [00:00<00:00, 10.95it/s]


In [31]:
# Validation
from lsdb import read_hipscat

read_hipscat("/astro/store/epyc3/data3/hats/tmp/sdss_ss_26/spectro/ss_26_catalog/sdss_dr7_spectra").compute()

,OBJID,HOLETYPE,RA,DEC,MAG,STARL,EXPL,DEVAUCL,OBJTYPE,XFOCAL,...,joined_OBJID,flux,ivar,andmask,ormask,wavedisp,skyflux,Norder,Dir,Npix
_hipscat_index,,,,,,,,,,,,,,,,,,,,,
7655344765885480960,[752 1 1 22 46],b'OBJECT',146.93358,-1.245374,[20.2916 19.921 19.6252 19.3595 19.1319],0.000013,0.0,0.0,b'QSO ',-174.34787,...,752112246,[6.78620243 6.78529406 6.78438807 ... 4.391211...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],0,0,6
7655345199949807616,[752 1 1 22 156],b'OBJECT',146.94621,-1.212941,[21.7005 20.123 18.9133 18.4487 18.0317],0.0,0.0,0.939723,b'GALAXY ',-171.56166,...,7521122156,[ 0.80459845 0.80449116 0.80438417 ... 11.23...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],0,0,6
7655361677990297600,[752 1 1 25 103],b'OBJECT',147.24633,-1.253256,[21.4659 19.7631 18.7353 18.2937 17.9048],0.0,0.0,0.058479,b'GALAXY ',-106.07909,...,7521125103,[ 4.57366419 4.57298851 4.57231426 ... 12.65...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],0,0,6
7655375118314504192,[752 1 1 26 96],b'OBJECT',147.404,-1.173301,[20.763 19.0641 18.1913 17.7117 17.3649],0.0,0.0,0.841395,b'GALAXY ',-71.699495,...,752112696,[13.5261488 13.52409649 13.52204704 ... 22.74...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],0,0,6
7655376147219218432,[752 1 1 26 108],b'OBJECT',147.47546,-1.181589,[21.9764 19.4901 18.1613 17.3037 16.862 ],0.0,0.0,0.0,b'GALAXY ',-56.132062,...,7521126108,[ 3.32240272 3.32188702 3.32137227 ... 34.66...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.],0,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7692993406418550784,[756 1 6 203 340],b'OBJECT',146.22688,1.245267,[22.9954 20.7788 19.2855 18.6784 18.2446],0.0,0.0,0.001567,b'GALAXY ',72.773317,...,75616203340,[3.45088482 3.45081496 3.4507463 ... 8.017875...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 17825792 17825...,[0. 0. 0. ... 0.772930...,[ 0. 0. 0. ... 9.76...,0,0,6
7692993406418550785,[756 1 6 203 340],b'OBJECT',146.22688,1.245267,[22.9954 20.7788 19.2855 18.6784 18.2446],0.0,0.0,0.001567,b'GALAXY ',72.773317,...,75616203340,[1.45147419 1.45134723 0.09362386 ... 7.400648...,[0. 0.10416706 0. ... 0. ...,[16777216 0 0 ... 16777216 16777...,[16777216 0 1048576 ... 16777216 16777...,[0. 1.19548416 1.19501197 ... 0.802508...,[0. 9.8389883 8.86986637 ... 5.706210...,0,0,6
7693274168002871296,[756 1 6 213 159],b'OBJECT',147.66407,1.210253,[20.4506 19.4843 19.1176 18.7854 18.718 ],0.0,0.000187,0.000178,b'GALAXY ',-15.051296,...,75616213159,[6.42660427 6.42617989 6.42575645 ... 4.521975...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0.819446...,[0. 0. 0. ... 6.618567...,0,0,6


In [63]:
# NEW -- use header 5 data as object table, store all spectral data as list-arrays

# Read Header 5 - 640 rows
obj_dat = Table.read("spectro/ss_26/0666/spPlate-0666-52149.fits", hdu=5, format='fits')
obj_dat.write('h5_pq/plate.parquet', format='parquet', overwrite=True)

# Read in spFrame HDU data
ndf = pd.read_parquet('h5_pq/plate.parquet', dtype_backend='pyarrow')
ndf["flux"] = pd.Series(spec[0].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["ivar"] = pd.Series(spec[1].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["andmask"] = pd.Series(spec[2].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
ndf["ormask"] = pd.Series(spec[3].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.int32())))
ndf["wavedisp"] = pd.Series(spec[4].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))
ndf["skyflux"] = pd.Series(spec[6].data.tolist(), dtype=pd.ArrowDtype(pa.list_(pa.float64())))

ndf


,OBJID,HOLETYPE,RA,DEC,MAG,STARL,EXPL,DEVAUCL,OBJTYPE,XFOCAL,...,SCI_EXPTIME,CALIBFLUX,CALIBFLUX_IVAR,SFD_EBV,flux,ivar,andmask,ormask,wavedisp,skyflux
0,[1729 0 2 290 91],b'OBJECT',31.523155,-9.756101,[23.2484 19.117 18.2746 17.746 17.6056],0.0,0.0,0.0,b'GALAXY ',199.74945,...,0.0,[ 5.404982 21.817234 77.35365 112.39978 1...,[4.0015516 0. 0.6940962 0.24724363 0...,0.023031,[ 9.42593384 9.426754 9.4275732 ... 14.39...,[0. 0. 0. ... 0. ...,[16777216 16777216 16777216 ... 0 ...,[16777216 16777216 16777216 ... 0 ...,[0. 0. 0. ... 0.711992...,[0. 0. 0. ... 6.316115...
1,[1729 0 1 289 132],b'OBJECT',31.469403,-10.083748,[19.8646 19.5346 19.23 19.2221 19.2284],0.737904,0.302691,0.347536,b'QSO ',188.15113,...,0.0,[15.067631 17.1941 23.961672 23.531322 24.63...,[4.6041636 9.9697075 3.592663 5.5618854 0.696...,0.021243,[10.50532627 10.50635529 10.5073843 ... 2.67...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0.713815...,[0. 0. 0. ... 6.233839...
2,[1893 0 1 21 305],b'OBJECT',31.488311,-9.818725,[22.8947 21.2806 19.6487 18.9896 18.6624],0.0,0.280543,0.559758,b'GALAXY ',192.24013,...,0.0,[ 0.72263724 3.8460758 17.426212 31.697823...,[ 0. 29.277908 11.148463 3.896464...,0.023524,[0.17190854 0.17192382 0.1719391 ... 4.806507...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0.711536...,[0. 0. 0. ... 6.364105...
3,[1893 0 1 21 115],b'OBJECT',31.456939,-9.822613,[20.3704 19.3222 19.1334 19.0496 20.4901],0.0,0.048529,0.0,b'GALAXY ',185.49547,...,0.0,[10.137507 24.42646 30.260962 32.72122 8.72...,[1.787897 3.9496963 2.793673 1.9914691 0...,0.023824,[13.54622364 13.54742813 13.54863167 ... 4.13...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0.712753...,[ 0. 0. 0. ... 6.08...
4,[1729 0 2 290 150],b'OBJECT',31.559744,-9.795424,[21.2908 19.3041 18.3566 17.909 17.5665],0.0,0.0,0.0,b'GALAXY ',207.60744,...,0.0,[ 4.0146995 22.381767 54.149 79.69047...,[4.3479543 9.688629 2.6090639 5.3602376 0...,0.022649,[10.67145443 10.67240047 10.67334366 ... 11.68...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0.712845...,[ 0. 0. 0. ... 6.50...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,[1729 0 5 289 88],b'OBJECT',31.217188,-8.500655,[21.0269 19.2317 18.4162 17.9991 17.7485],0.0,0.0,0.0,b'GALAXY ',134.44982,...,0.0,[ 5.232864 23.22778 51.007195 72.846756 92.63...,[ 4.7004595 23.414831 7.0491943 2.695927...,0.021539,[ 5.33581734 5.33555269 5.33528805 ... 13.26...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.]
636,[1729 0 4 288 137],b'OBJECT',31.107373,-8.886252,[21.5286 19.7775 18.7371 18.1799 17.8261],0.0,0.0,0.841395,b'GALAXY ',110.62351,...,0.0,[ 3.0970795 14.003801 37.69825 60.59455 8...,[ 4.521752 25.415453 7.961527 5.085474...,0.023625,[ 3.70717764 3.70704007 3.70690298 ... 11.75...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.]
637,[1893 0 4 19 45],b'OBJECT',31.079067,-8.735306,[18.0334 16.1465 15.3051 14.8439 14.5598],0.0,0.0,0.150661,b'GALAXY ',104.58962,...,0.0,[ 81.196686 431.04742 942.5549 1398.0271...,[1.3640302 2.0554833 0.7252578 0.45153797 0...,0.02393,[ 78.07817078 78.07486725 78.07156372 ... 28...,[0. 0. 0. ... 0. 0. 0.],[16777216 16777216 16777216 ... 16777216 16777...,[16777216 16777216 16777216 ... 16777216 16777...,[0. 0. 0. ... 0. 0. 0.],[0. 0. 0. ... 0. 0. 0.]
638,[1729 0 4 288 447],b'OBJECT',31.158741,-8.956442,[22.3167 21.7758 19.9561 19.1008 18.7426],0.0,0.0,0.895365,b'GALAXY ',

In [4]:
#OLD
# Read Object Table - 640 rows
obj_dat = Table.read("spectro/ss_26/0666/spObj-0666-52149-26.fit", hdu=1, format='fits')

#Convert to NestedFrame
obj_cols = ["run", "camcol", "rerun", "field", "parent", "id", "nchild", 
"ra","dec", "lambda", "eta", "l","b", "primtarget", "sectarget"]
ndf = npd.NestedFrame(obj_dat[obj_cols].to_pandas())

# Read in spFrame HDU data
with fits.open("spectro/ss_26/0666/spPlate-0666-52149.fits") as spec:
    ndf["flux"] = pd.Series(spec[0].data.tolist())
    ndf["ivar"] = pd.Series(spec[1].data.tolist())
    ndf["andmask"] = pd.Series(spec[2].data.tolist())
    ndf["ormask"] = pd.Series(spec[3].data.tolist())
    ndf["wavedisp"] = pd.Series(spec[4].data.tolist())
    ndf["skyflux"] = pd.Series(spec[6].data.tolist())

ndf_spec = npd.NestedFrame.from_lists(ndf, base_columns=obj_cols, name="spectra")
ndf_spec

,run,camcol,rerun,field,parent,id,nchild,ra,dec,lambda,eta,l,b,primtarget,sectarget,spectra
0,1729,2,40,290,166,167,0,31.523135,-9.756108,-153.889674,-21.622311,171.850705,-65.266138,64,0,flux ivar andmask ormask ...
1,1729,1,40,289,-1,103,0,31.469417,-10.083744,-153.970429,-21.263943,172.295789,-65.541507,2,0,flux ivar andmask ormask ...
2,1893,1,40,21,177,178,0,31.488321,-9.818716,-153.929148,-21.556075,171.892612,-65.335349,32,0,flux ivar andmask ormask ...
3,1893,1,0,21,112,115,0,31.456939,-9.822613,-153.960289,-21.554665,171.844690,-65.359262,64,0,flux ivar andmask ormask ...
4,1729,2,40,290,180,181,0,31.559715,-9.795413,-153.857076,-21.575296,171.977785,-65.270343,64,0,flux ivar andmask ormask ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,1729,5,40,289,-1,110,0,31.217162,-8.500660,-154.092386,-23.041297,169.347458,-64.531229,64,0,flux ivar andmask ormask wav...
636,1729,4,40,288,110,111,0,31.107377,-8.886263,-154.229163,-22.622962,169.746001,-64.893840,64,0,flux ivar andmask ormask wav...
637,1893,4,40,19,13,14,0,31.079039,-8.735293,-154.245796,-22.792451,169.463528,-64.798129,96,0,flux ivar andmask ormask wa...
638,1729,4,40,288,180,181,0,31.158739,-8.956450,-154.183893,-22.540958,169.944968,-64.913311,32,0,flux ivar andmask ormask wave...


In [16]:
ndf_spec["spectra"].iloc[0]

,flux,ivar,andmask,ormask,wavedisp,skyflux
0,9.425934,0.000000,16777216,16777216,0.000000,0.000000
1,9.426754,0.000000,16777216,16777216,0.000000,0.000000
2,9.427573,0.000000,16777216,16777216,0.000000,0.000000
3,9.428390,0.000000,16777216,16777216,0.000000,0.000000
4,9.429207,0.000000,16777216,16777216,0.000000,0.000000
...,...,...,...,...,...,...
3844,14.376133,1.317432,0,0,0.712094,5.603570
3845,13.377356,1.356143,0,0,0.712099,5.916445
3846,14.397409,0.000000,0,0,0.711992,6.316115
3847,15.417425,1.253372,0,0,0.712273,6.734169


In [5]:
ndf_spec.to_parquet("./plate_test", by_layer=True)

In [6]:
obj = fits.open("spectro/ss_26/0666/spObj-0666-52149-26.fit")
spec = fits.open("spectro/ss_26/0666/spPlate-0666-52149.fits")

In [12]:
spec[5].data[:2]

FITS_rec([([1729,    0,    2,  290,   91], 'OBJECT', 31.523155,  -9.7561011, [23.2484, 19.117 , 18.2746, 17.746 , 17.6056], 0.      , 0.      , 2.22331e-08, 'GALAXY', 199.74945, -163.01831, 1, 1, 2230, 64, 0, 1, 0., [  5.404982,  21.817234,  77.35365 , 112.39978 , 157.49948 ], [4.0015516 , 0.        , 0.6940962 , 0.24724363, 0.08935539], 0.02303132),
          ([1729,    0,    1,  289,  132], 'OBJECT', 31.469403, -10.083748 , [19.8646, 19.5346, 19.23  , 19.2221, 19.2284], 0.737904, 0.302691, 3.47536e-01, 'QSO', 188.15113, -234.50142, 1, 2, 2224,  4, 0, 1, 0., [ 15.067631,  17.1941  ,  23.961672,  23.531322,  24.633608], [4.6041636 , 9.9697075 , 3.592663  , 5.5618854 , 0.6966781 ], 0.0212433 )],
         dtype=(numpy.record, [('OBJID', '>i4', (5,)), ('HOLETYPE', 'S6'), ('RA', '>f8'), ('DEC', '>f8'), ('MAG', '>f4', (5,)), ('STARL', '>f4'), ('EXPL', '>f4'), ('DEVAUCL', '>f4'), ('OBJTYPE', 'S16'), ('XFOCAL', '>f8'), ('YFOCAL', '>f8'), ('SPECTROGRAPHID', '>i4'), ('FIBERID', '>i4'), ('THROUG

In [8]:
np.shape(spec[5].data)

(640,)

In [66]:
spec[0].data[0]

array([ 9.425934,  9.426754,  9.427573, ..., 14.397409, 15.417425,
       15.417142], dtype='>f4')

In [89]:
import matplotlib.pyplot as plt

len(spec[0].data[0])

3849

In [88]:
#Table.read.help('fits')

In [129]:
len(obj[1].data)

640

In [23]:
spec_dat = Table.read("spectro/ss_26/0666/spPlate-0666-52149.fits", hdu=5, format='fits')
spec_dat[:2].to_pandas()

ValueError: Cannot convert a table with multidimensional columns to a pandas DataFrame. Offending columns are: ['OBJID', 'MAG', 'CALIBFLUX', 'CALIBFLUX_IVAR']
One can filter out such columns using:
names = [name for name in tbl.colnames if len(tbl[name].shape) <= 1]
tbl[names].to_pandas(...)

In [22]:
spec[5].data[:2]

FITS_rec([([1729,    0,    2,  290,   91], 'OBJECT', 31.523155,  -9.7561011, [23.2484, 19.117 , 18.2746, 17.746 , 17.6056], 0.      , 0.      , 2.22331e-08, 'GALAXY', 199.74945, -163.01831, 1, 1, 2230, 64, 0, 1, 0., [  5.404982,  21.817234,  77.35365 , 112.39978 , 157.49948 ], [4.0015516 , 0.        , 0.6940962 , 0.24724363, 0.08935539], 0.02303132),
          ([1729,    0,    1,  289,  132], 'OBJECT', 31.469403, -10.083748 , [19.8646, 19.5346, 19.23  , 19.2221, 19.2284], 0.737904, 0.302691, 3.47536e-01, 'QSO', 188.15113, -234.50142, 1, 2, 2224,  4, 0, 1, 0., [ 15.067631,  17.1941  ,  23.961672,  23.531322,  24.633608], [4.6041636 , 9.9697075 , 3.592663  , 5.5618854 , 0.6966781 ], 0.0212433 )],
         dtype=(numpy.record, [('OBJID', '>i4', (5,)), ('HOLETYPE', 'S6'), ('RA', '>f8'), ('DEC', '>f8'), ('MAG', '>f4', (5,)), ('STARL', '>f4'), ('EXPL', '>f4'), ('DEVAUCL', '>f4'), ('OBJTYPE', 'S16'), ('XFOCAL', '>f8'), ('YFOCAL', '>f8'), ('SPECTROGRAPHID', '>i4'), ('FIBERID', '>i4'), ('THROUG

In [79]:
spec.info()

Filename: spectro/ss_26/0666/spPlate-0666-52149.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU     199   (3849, 640)   float32   
  1                1 ImageHDU        15   (3849, 640)   float32   
  2                1 ImageHDU        14   (3849, 640)   int32   
  3                1 ImageHDU        14   (3849, 640)   int32   
  4                1 ImageHDU        15   (3849, 640)   float32   
  5                1 BinTableHDU     59   640R x 21C   ['5J', '6A', 'D', 'D', '5E', 'E', 'E', 'E', '16A', 'D', 'D', 'J', 'J', 'J', 'J', 'J', 'J', 'E', '5E', '5E', 'E']   
  6                1 ImageHDU         7   (3849, 640)   float32   
  7                1 ImageHDU         7   (5, 640)   float32   
  8                1 ImageHDU         7   (3, 640)   float32   


In [133]:
spec[0].header

SIMPLE  =                    T / FIT STANDARD                                   
BITPIX  =                  -32 / FITS BITS/PIXEL                                
NAXIS   =                    2 / NUMBER OF AXES                                 
NAXIS1  =                 3849 / WIDTH (TOTAL!)                                 
NAXIS2  =                  640 / HEIGHT (TOTAL!)                                
EXTEND  =                    T /                                                
TAI     =        4505617383.37 / 1st row - Number of seconds since Nov 17 1858  
RA      =            30.592416 / 1st row - Right ascension of telescope boresigh
DEC     =           -9.008748  / 1st row - Declination of telescope boresight (d
EQUINOX =              2000.00 /                                                
RADECSYS= 'FK5     '           /                                                
AZ      =        272.914059222 / 1st row - Azimuth  (encoder) of tele (0=N?) (de
ALT     =        47.63957600

In [123]:
len(spec[6].data[0])

3849

In [132]:
obj_dat = Table.read("spectro/ss_26/0666/spObj-0666-52149-26.fit", hdu=1, format='fits')
obj_dat[0:2]

run,camcol,rerun,field,parent,id,nchild,objc_type,objc_prob_psf,catid,objc_flags,objc_flags2,objc_rowc,objc_rowcerr,objc_colc,objc_colcerr,rowv,rowverr,colv,colverr,rowc,rowcerr,colc,colcerr,sky,skyerr,psfcounts,psfcountserr,fibercounts,fibercountserr,petrocounts,petrocountserr,petrorad,petroraderr,petror50,petror50err,petror90,petror90err,q,qerr,u,uerr,m_e1,m_e2,m_e1e1err,m_e1e2err,m_e2e2err,m_rr_cc,m_rr_ccerr,m_cr4,m_e1_psf,m_e2_psf,m_rr_cc_psf,m_cr4_psf,iso_rowc,iso_rowcerr,iso_rowcgrad,iso_colc,iso_colcerr,iso_colcgrad,iso_a,iso_aerr,iso_agrad,iso_b,iso_berr,iso_bgrad,iso_phi,iso_phierr,iso_phigrad,r_dev,r_deverr,ab_dev,ab_deverr,phi_dev,phi_deverr,counts_dev,counts_deverr,r_exp,r_experr,ab_exp,ab_experr,phi_exp,phi_experr,counts_exp,counts_experr,counts_model,counts_modelerr,texture,star_l,star_lnl,exp_l,exp_lnl,dev_l,dev_lnl,fracpsf,flags,flags2,type,prob_psf,nprof,profmean,proferr,status,ra,dec,lambda,eta,l,b,offsetra,offsetdec,primtarget,sectarget,reddening,propermotionmatch,propermotiondelta,propermotion,propermotionangle,usnoblue,usnored,firstmatch,firstid,firstlambda,firsteta,firstdelta,firstpeak,firstint,firstrms,firstmajor,firstminor,firstpa,rosatmatch,rosatdelta,rosatposerr,rosatcps,rosatcpserr,rosathr1,rosathr1err,rosathr2,rosathr2err,rosatext,rosatextlike,rosatdetectlike,rosatexposure,priority,matchid,sprerun,vers_2d,vers_1d,plate,mjd,mapid,fiberid,objid,diag_primtarget,diag_sectarget,xfocal,yfocal,diag_ra,diag_dec,skyres,ugrizfibre,griflux,grisn,deriv2,mean,contchi2,blueslope,redslope,class,zstat,zfinal,zerrfinal,zconffinal,zxcorr,zerrxcorr,zconfxcorr,zemz,zerremz,zconfemz,templateid,nemline,zwarning,galclass,vel_dis,vel_dise,eclass,ecoeff1,ecoeff2,ecoeff3,ecoeff4,ecoeff5,plugmapobj_objid,holetype,plugmapobj_ra,plugmapobj_dec,mag,starl,expl,devaucl,objtype,plugmapobj_xfocal,plugmapobj_yfocal,spectrographid,plugmapobj_fiberid,throughput,plugmapobj_primtarget,plugmapobj_sectarget
int32,int32,int32,int32,int32,int32,int32,int32,float32,int32,int32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],float32[5],int32[5],int32[5],int32[5],float32[5],int32[5],"float32[5,15]","float32[5,15]",int32,float64,float64,float64,float64,float64,float64,float32[5],float32[5],int32,int32,float32[5],int32,float32,float32,float32,float32,float32,int32,int32,float64,float64,float32,float32,float32,float32,float32,float32,float32,int32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,"int32[10,5]",int32,bytes19,bytes19,int32,int32,bytes19,int32,int32[5],int32,int32,float64,float64,float64,float64,float32[3],float32[5],float32[3],float32[3],float32,float32,float32,float32,float32,bytes80,bytes80,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,int32,int32,float32,float64,float64,float64,float64,float64,float64,float64,float64,int32[5],bytes80,float64,float64,float32[5],float32,float32,float32,bytes80,float64,float64,int32,int32,int32,int32,int32
1729,2,40,290,166,167,0,3,0.0,0,268566544,36864,755.6808,0.01255912,543.89215,0.019416926,0.0032394822,0.0058910768,0.0018419423,0.005908007,760.37775 .. 761.90576,0.2198638 .. 0.033098206,547.1